# 🔥 FILE 2-A: Dataset, DataLoader & Training Loop

**PHẦN 2 - INTERMEDIATE (CORE DEEP LEARNING)**

---

## 📋 Nội Dung

✅ Dataset là gì và tại sao cần thiết

✅ Custom Dataset - Tạo dataset riêng

✅ DataLoader - Load data hiệu quả

✅ Train/Validation/Test split

✅ Training loop chuẩn với validation

✅ Overfitting vs Underfitting

✅ Early Stopping

✅ Dự án hoàn chỉnh: Classification pipeline

---

## 🎯 Mục Tiêu

Sau file này, bạn sẽ:
- Tạo được custom Dataset cho bất kỳ data nào
- Sử dụng DataLoader hiệu quả
- Viết training loop production-ready
- Hiểu và detect overfitting/underfitting
- Implement early stopping

---

## ⏱️ Thời Gian Học: 3-4 giờ

---

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

print(f"PyTorch version: {torch.__version__}")
print("✅ Ready to learn Dataset & DataLoader!")

---

# 1️⃣ Dataset Là Gì?

## Vấn Đề Với Cách Cũ

Ở Foundation level, chúng ta load toàn bộ data vào RAM:

```python
X = torch.randn(1000, 10)  # Load hết vào RAM
y = torch.randn(1000, 1)

for epoch in range(epochs):
    y_pred = model(X)  # Process toàn bộ cùng lúc
    loss = criterion(y_pred, y)
    ...
```

### Vấn Đề:

❌ **Không scale**: Dữ liệu lớn (GB, TB) không fit vào RAM

❌ **Không shuffle**: Data không được trộn

❌ **Không batch**: Phải process toàn bộ cùng lúc

❌ **Không parallel**: Không load data song song

## Giải Pháp: Dataset + DataLoader

### Dataset
- Định nghĩa **cách load một sample**
- Load on-demand (chỉ khi cần)
- Support nhiều định dạng (images, text, audio)

### DataLoader
- **Batch**: Chia data thành batches
- **Shuffle**: Trộn data mỗi epoch
- **Parallel**: Load data song song (multi-processing)
- **Memory efficient**: Chỉ load batch hiện tại

---

# 2️⃣ Custom Dataset

## Cấu Trúc Dataset

In [ ]:
print("=" * 70)
print("CUSTOM DATASET - TEMPLATE")
print("=" * 70)

class CustomDataset(Dataset):
    """Template cho custom dataset"""
    
    def __init__(self, data, targets):
        """
        Constructor - Khởi tạo dataset
        Args:
            data: Input data
            targets: Labels/targets
        """
        self.data = data
        self.targets = targets
    
    def __len__(self):
        """
        Trả về số lượng samples
        BẮT BUỘC implement!
        """
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        Trả về sample tại index idx
        BẮT BUỘC implement!
        Args:
            idx: Index của sample
        Returns:
            (data, target) tuple
        """
        x = self.data[idx]
        y = self.targets[idx]
        return x, y

print("""
3 PHƯƠNG THỨC BẮT BUỘC:

1. __init__(self, ...)
   → Khởi tạo dataset, lưu paths, metadata

2. __len__(self)
   → Trả về tổng số samples
   → DataLoader dùng để biết khi nào hết data

3. __getitem__(self, idx)
   → Load và trả về sample tại index idx
   → DataLoader gọi method này để lấy data
""")

## Ví Dụ Thực Tế

In [ ]:
print("=" * 70)
print("VÍ DỤ: REGRESSION DATASET")
print("=" * 70)

class RegressionDataset(Dataset):
    """Dataset cho regression task"""
    
    def __init__(self, X, y, transform=None):
        self.X = torch.FloatTensor(X)
        self.y = torch.FloatTensor(y)
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx]
        
        # Apply transform nếu có
        if self.transform:
            x = self.transform(x)
        
        return x, y

# Tạo dữ liệu mẫu
X_data = np.random.randn(1000, 10)
y_data = np.random.randn(1000, 1)

# Tạo dataset
dataset = RegressionDataset(X_data, y_data)

print(f"\nDataset size: {len(dataset)}")
print(f"\nFirst sample:")
x, y = dataset[0]
print(f"  X: shape={x.shape}, dtype={x.dtype}")
print(f"  y: shape={y.shape}, dtype={y.dtype}")

# Access multiple samples
print(f"\nSamples 0-2:")
for i in range(3):
    x, y = dataset[i]
    print(f"  Sample {i}: X={x[:3]}..., y={y.item():.4f}")

In [ ]:
print("=" * 70)
print("VÍ DỤ: CLASSIFICATION DATASET")
print("=" * 70)

class ClassificationDataset(Dataset):
    """Dataset cho classification task"""
    
    def __init__(self, features, labels):
        self.features = torch.FloatTensor(features)
        self.labels = torch.LongTensor(labels)  # LongTensor cho classification
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Tạo dữ liệu classification
X, y = make_classification(n_samples=1000, n_features=20, 
                           n_informative=15, n_redundant=5,
                           n_classes=3, random_state=42)

dataset = ClassificationDataset(X, y)

print(f"\nDataset info:")
print(f"  Total samples: {len(dataset)}")
print(f"  Feature dim: {dataset.features.shape[1]}")
print(f"  Number of classes: {len(torch.unique(dataset.labels))}")

# Check class distribution
unique, counts = torch.unique(dataset.labels, return_counts=True)
print(f"\nClass distribution:")
for class_id, count in zip(unique, counts):
    print(f"  Class {class_id}: {count} samples ({count/len(dataset)*100:.1f}%)")

---

# 3️⃣ DataLoader

## DataLoader Là Gì?

In [ ]:
print("=" * 70)
print("DATALOADER BASICS")
print("=" * 70)

# Tạo dataset
X = np.random.randn(100, 5)
y = np.random.randint(0, 3, 100)
dataset = ClassificationDataset(X, y)

# Tạo DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=16,      # Mỗi batch 16 samples
    shuffle=True,       # Shuffle mỗi epoch
    num_workers=0,      # Number of subprocesses (0 = main process)
    drop_last=False     # Drop batch cuối nếu không đủ size
)

print(f"\nDataLoader info:")
print(f"  Dataset size: {len(dataset)}")
print(f"  Batch size: {dataloader.batch_size}")
print(f"  Number of batches: {len(dataloader)}")
print(f"  Shuffle: {dataloader.shuffle}")

# Iterate through batches
print(f"\nFirst 3 batches:")
for i, (batch_X, batch_y) in enumerate(dataloader):
    if i >= 3:
        break
    print(f"  Batch {i}: X={batch_X.shape}, y={batch_y.shape}")

print("\n💡 DataLoader tự động:")
print("  - Chia thành batches")
print("  - Shuffle (nếu shuffle=True)")
print("  - Convert sang tensor")
print("  - Collate (gom samples thành batch)")

## DataLoader Parameters

In [ ]:
print("=" * 70)
print("DATALOADER PARAMETERS")
print("=" * 70)

print("""
QUAN TRỌNG:

1. batch_size (int)
   → Số samples mỗi batch
   → Lớn hơn = nhanh hơn nhưng tốn RAM
   → Thường: 16, 32, 64, 128, 256

2. shuffle (bool)
   → True: Trộn data mỗi epoch
   → QUAN TRỌNG cho training!
   → False cho validation/test

3. num_workers (int)
   → Số processes load data song song
   → 0 = main process (đơn giản, dễ debug)
   → >0 = faster nhưng phức tạp hơn
   → Khuyến nghị: 0-4

4. drop_last (bool)
   → True: Bỏ batch cuối nếu không đủ size
   → False: Giữ lại (batch cuối nhỏ hơn)

5. pin_memory (bool)
   → True: Tăng tốc CPU → GPU transfer
   → Chỉ dùng khi train trên GPU
""")

# Ví dụ với các parameters khác nhau
print("\n" + "=" * 70)
print("SO SÁNH CONFIGURATIONS")
print("=" * 70)

configs = [
    {'batch_size': 32, 'shuffle': True, 'drop_last': False},
    {'batch_size': 32, 'shuffle': True, 'drop_last': True},
    {'batch_size': 16, 'shuffle': False, 'drop_last': False},
]

for i, config in enumerate(configs):
    loader = DataLoader(dataset, **config)
    print(f"\nConfig {i+1}: {config}")
    print(f"  → {len(loader)} batches")
    last_batch_size = len(dataset) % config['batch_size']
    if last_batch_size > 0:
        print(f"  → Last batch: {last_batch_size if not config['drop_last'] else 'dropped'} samples")

---

# 4️⃣ Train/Validation/Test Split

In [ ]:
print("=" * 70)
print("TRAIN / VALIDATION / TEST SPLIT")
print("=" * 70)

print("""
TẠI SAO CẦN CHIA?

TRAINING SET (60-80%):
  → Dùng để train model
  → Update weights/parameters
  → Optimize loss

VALIDATION SET (10-20%):
  → Dùng để tune hyperparameters
  → Select best model
  → Early stopping
  → KHÔNG dùng để train!

TEST SET (10-20%):
  → Đánh giá final performance
  → CHỈ dùng MỘT LẦN cuối cùng
  → Simulate real-world performance
""")

# Tạo dữ liệu
X, y = make_classification(n_samples=1000, n_features=20, 
                           n_classes=3, random_state=42)

# CÁCH 1: Sklearn train_test_split
print("\n" + "=" * 70)
print("CÁCH 1: Sklearn train_test_split")
print("=" * 70)

# Train/Temp split (80/20)
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Val/Test split (50/50 of temp = 10/10 of total)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"\nData split:")
print(f"  Train: {len(X_train)} samples ({len(X_train)/len(X)*100:.0f}%)")
print(f"  Val:   {len(X_val)} samples ({len(X_val)/len(X)*100:.0f}%)")
print(f"  Test:  {len(X_test)} samples ({len(X_test)/len(X)*100:.0f}%)")

# Tạo datasets
train_dataset = ClassificationDataset(X_train, y_train)
val_dataset = ClassificationDataset(X_val, y_val)
test_dataset = ClassificationDataset(X_test, y_test)

print(f"\n✅ Datasets created")

In [ ]:
# CÁCH 2: PyTorch random_split
print("=" * 70)
print("CÁCH 2: PyTorch random_split")
print("=" * 70)

# Tạo full dataset
full_dataset = ClassificationDataset(X, y)

# Define sizes
train_size = int(0.8 * len(full_dataset))
val_size = int(0.1 * len(full_dataset))
test_size = len(full_dataset) - train_size - val_size

# Random split
train_dataset, val_dataset, test_dataset = random_split(
    full_dataset, 
    [train_size, val_size, test_size],
    generator=torch.Generator().manual_seed(42)  # For reproducibility
)

print(f"\nSplit sizes:")
print(f"  Train: {len(train_dataset)}")
print(f"  Val:   {len(val_dataset)}")
print(f"  Test:  {len(test_dataset)}")

print("\n💡 Sklearn vs PyTorch:")
print("  Sklearn: Có stratify (giữ class balance)")
print("  PyTorch: Đơn giản, random thuần")

In [ ]:
# Tạo DataLoaders
print("=" * 70)
print("TẠO DATALOADERS")
print("=" * 70)

batch_size = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,        # ← SHUFFLE cho training
    num_workers=0
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,       # ← KHÔNG shuffle cho val
    num_workers=0
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,       # ← KHÔNG shuffle cho test
    num_workers=0
)

print(f"\nDataLoaders:")
print(f"  Train: {len(train_loader)} batches")
print(f"  Val:   {len(val_loader)} batches")
print(f"  Test:  {len(test_loader)} batches")

print("\n⚠️ QUAN TRỌNG:")
print("  - Training: shuffle=True")
print("  - Val/Test: shuffle=False")

---

# 5️⃣ Training Loop Chuẩn Với Validation

In [ ]:
print("=" * 70)
print("TRAINING LOOP TEMPLATE")
print("=" * 70)

# Setup device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}\n")

# Model
class Classifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = Classifier(input_dim=20, hidden_dim=64, num_classes=3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Model created")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
print("=" * 70)
print("TRAINING WITH VALIDATION")
print("=" * 70)

def train_epoch(model, train_loader, criterion, optimizer, device):
    """Train một epoch"""
    model.train()  # Set to training mode
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_X, batch_y in train_loader:
        # Move to device
        batch_X = batch_X.to(device)
        batch_y = batch_y.to(device)
        
        # Forward
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Statistics
        running_loss += loss.item() * batch_X.size(0)
        _, predicted = torch.max(outputs, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc


def validate_epoch(model, val_loader, criterion, device):
    """Validate một epoch"""
    model.eval()  # Set to evaluation mode
    
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():  # Không tính gradient
        for batch_X, batch_y in val_loader:
            batch_X = batch_X.to(device)
            batch_y = batch_y.to(device)
            
            # Forward only
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            
            # Statistics
            running_loss += loss.item() * batch_X.size(0)
            _, predicted = torch.max(outputs, 1)
            total += batch_y.size(0)
            correct += (predicted == batch_y).sum().item()
    
    epoch_loss = running_loss / total
    epoch_acc = correct / total
    
    return epoch_loss, epoch_acc

print("✅ Training functions defined")

In [ ]:
# Main training loop
print("=" * 70)
print("TRAINING")
print("=" * 70)

num_epochs = 50
history = {
    'train_loss': [],
    'train_acc': [],
    'val_loss': [],
    'val_acc': []
}

print(f"\n{'Epoch':>6} {'Train Loss':>12} {'Train Acc':>12} {'Val Loss':>12} {'Val Acc':>12}")
print("-" * 60)

for epoch in range(num_epochs):
    # Train
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # Validate
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    
    # Print progress
    if (epoch + 1) % 5 == 0:
        print(f"{epoch+1:6d} {train_loss:12.4f} {train_acc:12.2%} {val_loss:12.4f} {val_acc:12.2%}")

print("\n✅ Training completed!")

In [ ]:
# Visualize training
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot accuracy
axes[1].plot(history['train_acc'], label='Train Acc', linewidth=2)
axes[1].plot(history['val_acc'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Results:")
print(f"  Train Loss: {history['train_loss'][-1]:.4f}")
print(f"  Train Acc:  {history['train_acc'][-1]:.2%}")
print(f"  Val Loss:   {history['val_loss'][-1]:.4f}")
print(f"  Val Acc:    {history['val_acc'][-1]:.2%}")

---

# 6️⃣ Overfitting vs Underfitting

## Định Nghĩa

In [ ]:
print("=" * 70)
print("OVERFITTING VS UNDERFITTING")
print("=" * 70)

print("""
UNDERFITTING (Học kém):
  📉 Train loss: CAO
  📉 Val loss: CAO
  📉 Gap: NHỎ
  
  → Model quá đơn giản
  → Không học được pattern
  
  Giải pháp:
  ✅ Tăng model complexity
  ✅ Train lâu hơn
  ✅ Giảm regularization

---

GOOD FIT (Vừa đủ):
  📊 Train loss: THẤP
  📊 Val loss: THẤP
  📊 Gap: NHỎ
  
  → Model học tốt
  → Generalize tốt
  
  ✅ Mục tiêu cần đạt!

---

OVERFITTING (Học vẹt):
  📈 Train loss: RẤT THẤP
  📉 Val loss: CAO
  📈 Gap: LỚN
  
  → Model nhớ data training
  → Không generalize
  
  Giải pháp:
  ✅ Thêm data
  ✅ Regularization (Dropout, Weight Decay)
  ✅ Giảm model complexity
  ✅ Early stopping
  ✅ Data augmentation
""")

## Demo Overfitting

In [ ]:
print("=" * 70)
print("DEMO: TẠO OVERFITTING")
print("=" * 70)

# Tạo data NHỎ để dễ overfit
X_small, y_small = make_classification(
    n_samples=200,      # ← ÍT data
    n_features=20,
    n_informative=10,
    n_classes=3,
    random_state=42
)

X_train, X_val, y_train, y_val = train_test_split(
    X_small, y_small, test_size=0.3, random_state=42
)

train_ds = ClassificationDataset(X_train, y_train)
val_ds = ClassificationDataset(X_val, y_val)

train_ld = DataLoader(train_ds, batch_size=16, shuffle=True)
val_ld = DataLoader(val_ds, batch_size=16, shuffle=False)

# Model RẤT LỚN so với data
class LargeModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(20, 256)   # ← RẤT LỚN
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 3)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.fc4(x)
        return x

large_model = LargeModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(large_model.parameters(), lr=0.001)

print(f"\nModel size: {sum(p.numel() for p in large_model.parameters()):,} parameters")
print(f"Data size: {len(train_ds)} training samples")
print(f"Ratio: {sum(p.numel() for p in large_model.parameters()) / len(train_ds):.1f} params/sample")
print("\n⚠️ Model quá lớn so với data → SẼ OVERFIT!")

In [ ]:
# Train để thấy overfitting
num_epochs = 100
overfit_history = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

print("Training (sẽ overfit)...\n")

for epoch in range(num_epochs):
    train_loss, train_acc = train_epoch(large_model, train_ld, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(large_model, val_ld, criterion, device)
    
    overfit_history['train_loss'].append(train_loss)
    overfit_history['val_loss'].append(val_loss)
    overfit_history['train_acc'].append(train_acc)
    overfit_history['val_acc'].append(val_acc)
    
    if (epoch + 1) % 10 == 0:
        gap = train_loss - val_loss
        print(f"Epoch {epoch+1:3d}: Train={train_loss:.4f}, Val={val_loss:.4f}, Gap={gap:+.4f}")

print("\n✅ Training done")

In [ ]:
# Visualize overfitting
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(overfit_history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(overfit_history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Loss', fontsize=12)
axes[0].set_title('Overfitting: Loss Curves', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Annotate overfitting region
overfit_start = 20
axes[0].axvline(x=overfit_start, color='r', linestyle='--', alpha=0.5)
axes[0].text(overfit_start+5, max(overfit_history['val_loss'])*0.8, 
             'Overfitting\nstarts here', fontsize=10, color='red')

# Accuracy
axes[1].plot(overfit_history['train_acc'], label='Train Acc', linewidth=2)
axes[1].plot(overfit_history['val_acc'], label='Val Acc', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Accuracy', fontsize=12)
axes[1].set_title('Overfitting: Accuracy Curves', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 DẤU HIỆU OVERFITTING:")
print(f"  Train loss giảm tiếp: {overfit_history['train_loss'][-1]:.4f}")
print(f"  Val loss TĂNG lại: {overfit_history['val_loss'][-1]:.4f}")
print(f"  Gap LỚN: {abs(overfit_history['train_loss'][-1] - overfit_history['val_loss'][-1]):.4f}")
print(f"  Train acc = {overfit_history['train_acc'][-1]:.2%}")
print(f"  Val acc = {overfit_history['val_acc'][-1]:.2%}")

---

# 7️⃣ Early Stopping

In [ ]:
print("=" * 70)
print("EARLY STOPPING")
print("=" * 70)

class EarlyStopping:
    """Early stopping để tránh overfitting"""
    
    def __init__(self, patience=7, min_delta=0, verbose=True):
        """
        Args:
            patience: Số epochs chờ đợi trước khi stop
            min_delta: Minimum thay đổi để coi là improvement
            verbose: In ra thông tin
        """
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
    
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            if self.verbose:
                print(f"Initial best loss: {val_loss:.4f}")
        
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter}/{self.patience}")
            
            if self.counter >= self.patience:
                self.early_stop = True
        
        else:
            self.best_loss = val_loss
            self.counter = 0
            if self.verbose:
                print(f"New best loss: {val_loss:.4f}")

print("""
CÁCH HOẠT ĐỘNG:

1. Track val_loss mỗi epoch
2. Nếu val_loss KHÔNG cải thiện sau `patience` epochs
   → Dừng training
3. Save best model (val_loss thấp nhất)

LỢI ÍCH:
✅ Tránh overfitting
✅ Tiết kiệm thời gian
✅ Tự động tìm số epochs tối ưu
""")

In [ ]:
# Training với Early Stopping
print("=" * 70)
print("TRAINING VỚI EARLY STOPPING")
print("=" * 70)

# Reset model
model_es = Classifier(20, 64, 3).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_es.parameters(), lr=0.001)

early_stopping = EarlyStopping(patience=10, verbose=True)
history_es = {'train_loss': [], 'val_loss': [], 'train_acc': [], 'val_acc': []}

print("\nTraining...\n")

for epoch in range(200):  # Max 200 epochs
    train_loss, train_acc = train_epoch(model_es, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate_epoch(model_es, val_loader, criterion, device)
    
    history_es['train_loss'].append(train_loss)
    history_es['val_loss'].append(val_loss)
    history_es['train_acc'].append(train_acc)
    history_es['val_acc'].append(val_acc)
    
    print(f"Epoch {epoch+1}: Train Loss={train_loss:.4f}, Val Loss={val_loss:.4f}")
    
    # Early stopping check
    early_stopping(val_loss)
    
    if early_stopping.early_stop:
        print(f"\n⏹️ Early stopping triggered at epoch {epoch+1}")
        break
    
    if (epoch + 1) % 10 == 0:
        print()

print(f"\n✅ Training stopped at epoch {epoch+1}")
print(f"Best val loss: {early_stopping.best_loss:.4f}")

---

# ✅ Tổng Kết FILE 2-A

## Bạn Đã Học Được

✅ **Dataset** - Custom dataset cho bất kỳ data nào

✅ **DataLoader** - Batch, shuffle, parallel loading

✅ **Train/Val/Test split** - Chia data đúng cách

✅ **Training loop chuẩn** - Với validation

✅ **Overfitting vs Underfitting** - Nhận biết và xử lý

✅ **Early Stopping** - Tự động dừng khi overfit

---

## Template Code - Sử Dụng Ngay

```python
# 1. Dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 2. DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 3. Training Loop
for epoch in range(num_epochs):
    # Train
    model.train()
    for batch_X, batch_y in train_loader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # Validate
    model.eval()
    with torch.no_grad():
        for batch_X, batch_y in val_loader:
            outputs = model(batch_X)
            val_loss = criterion(outputs, batch_y)
```

---

## Bài Tập Thực Hành

### Bài 1: Custom Dataset
1. Tạo dataset cho regression với 1000 samples
2. Implement __init__, __len__, __getitem__
3. Add normalization transform
4. Test với DataLoader

### Bài 2: Complete Pipeline
1. Load iris dataset
2. Split 70/15/15 (train/val/test)
3. Create DataLoaders
4. Train với validation
5. Plot curves
6. Final test evaluation

### Bài 3: Detect Overfitting
1. Tạo small dataset (100 samples)
2. Build large model
3. Train và observe overfitting
4. Implement early stopping
5. Compare với/không early stopping

---

## Tiếp Theo

📗 **FILE 2-B: Optimizer, Activation & Regularization**

Bạn sẽ học:
- Advanced activation functions
- Optimizers chi tiết (SGD, Adam, AdamW)
- Learning rate scheduling
- Regularization (Dropout, Weight Decay, BatchNorm)

---